# Token Classification for Persian Kasr-e-Ezafeh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the dataset and parsing the data

In [ ]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)
    cnt = 0
    raw_text = file_path.read_text('utf-8').strip()
    cnt = 0
    tokens = []
    tags = []
    token_docs = []
    tag_docs = []
    for line in raw_text.split('\n'):
        token, tag = line.split('\t')
        if token == '.':
            cnt += 1
            
        tokens.append(token)
        tags.append(tag)
        if cnt == 3:
            token_docs.append(tokens)
            tag_docs.append(tags)
            cnt = 0
            tokens = []
            tags = []

    return token_docs, tag_docs

texts, tags = read_wnut('drive/MyDrive/train_data.txt')

In [ ]:
print(f"sample words: {texts[0][:10]}", f"sample tags: {tags[0][:10]}", sep='\n')
print(f"Total Words: {len(texts)}")

sample words: ['#', 'منبع', ':', ')', 'مجله', 'سروش', 'هفتگی', '،', 'مصاحبه', 'با']
sample tags: ['O', 'O', 'O', 'O', 'ye', 'e', 'O', 'O', 'O', 'O']
Total Words: 88571


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.1, train_size=.5)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
unique_tags

{'@e', 'O', 'e', 've', 'y', 'ye'}

In [ ]:
id2tag

{0: '@e', 1: 've', 2: 'ye', 3: 'y', 4: 'O', 5: 'e'}

## Preprocessing the data

In [ ]:
# !pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("HooshvareLab/distilbert-fa-zwnj-base")
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Encoding the tags

In [ ]:
label_all_tokens = True
import numpy as np

def encode_tags(tags, tokenized_inputs):
    labels = []
    tag_labels = [[tag2id[tag] for tag in doc] for doc in tags]
    for i, label in enumerate(tag_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    return labels



In [ ]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

In [ ]:
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained("HooshvareLab/distilbert-fa-zwnj-base", num_labels=len(unique_tags))

Some weights of the model checkpoint at HooshvareLab/distilbert-fa-zwnj-base were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/distilbert-fa-zwnj-base and are newly initialized: ['classifier.weight', 'classifier.bias']
Y

In [ ]:
from transformers import Trainer, TrainingArguments


In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [ ]:
# !pip install seqeval
# !pip install datasets
from datasets import load_metric

metric = load_metric("seqeval")

In [ ]:
u_tags =list(unique_tags)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [u_tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [u_tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 44285
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8304


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.048700,0.043130,0.921995,0.939136,0.930487,0.985994
2,0.038400,0.040256,0.936934,0.942790,0.939853,0.987362
3,0.024200,0.041852,0.935902,0.945430,0.940642,0.987561


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8858
  Batch size = 64
/usr/local/lib/python3.7/di

TrainOutput(global_step=8304, training_loss=0.05229067529133577, metrics={'train_runtime': 6469.8784, 'train_samples_per_second': 20.534, 'train_steps_per_second': 1.283, 'total_flos': 1.735917301435392e+16, 'train_loss': 0.05229067529133577, 'epoch': 3.0})

In [ ]:
res = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 8858
  Batch size = 64


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: e seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ye seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ve seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: y seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: @e seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'epoch': 3.0,
 'eval_accuracy': 0.9871010103354375,
 'eval_f1': 0.9377926803983672,
 'eval_loss': 0.04349938780069351,
 'eval_precision': 0.9341261928500214,
 'eval_recall': 0.9414880636223999,
 'eval_runtime': 181.5843,
 'eval_samples_per_second': 48.782,
 'eval_steps_per_second': 0.765}

In [ ]:
res